In [ ]:
import nomenclature
import ixmp4

In [ ]:
definition = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
platform = ixmp4.Platform("unep-fi-internal")

In [ ]:
supply_fuel = ["Electricity", "Heat"] # "Hydrogen" emissions for energy supply are not meaningful
species = ["Kyoto Gases", "CO2"] # "Kyoto Gases" for energy supply are only reported by the IAM-COMPACT scenarios

In [ ]:
def replace_tag(item, s, f):
    if pyam.utils.is_list_like(item):
        return [x.replace("{species}", s).replace("{supply_fuel}", f) for x in item]
    return item.replace("{species}", s).replace("{supply_fuel}", f)

In [ ]:
tagged_intensity_args =  {

# energy supply sectors
    "Emission Intensity|{species}|Energy|Supply|{supply_fuel}":
        (
            "Emissions|{species}|Energy|Supply|{supply_fuel}",
            "Secondary Energy|{supply_fuel}",
        ),

# iron and steel sector
    "Energy Intensity|Industry|Iron and Steel":
        (
            [
                "Final Energy|Industry|Iron and Steel",
                "Final Energy|Non-Energy Use|Iron and Steel",
            ],
            "Production|Iron and Steel|Steel",
        ),
    "Emission Intensity|CO2|Energy|Demand|Industry|Iron and Steel":
        (
            "Emissions|CO2|Energy|Demand|Industry|Iron and Steel",
             "Production|Iron and Steel|Steel",
        ),
    "Emission Intensity|CO2|Industrial Processes|Iron and Steel":
        (
            "Emissions|CO2|Industrial Processes|Iron and Steel",
            "Production|Iron and Steel|Steel",
        ),
    "Emission Intensity|CO2|Energy and Industrial Processes|Iron and Steel":
        (
            [
                "Emissions|CO2|Energy|Demand|Industry|Iron and Steel",
                "Emissions|CO2|Industrial Processes|Iron and Steel",
            ],
             "Production|Iron and Steel|Steel",
        ),

# chemicals sector
#    "Energy Intensity|Industry|Chemicals":
#        (
#            [
#                "Final Energy|Industry|Chemicals",
#                "Final Energy|Non-Energy Use|Chemicals",
#            ],
#            "Production|Chemicals",
#        ),
#    "Emission Intensity|CO2|Energy|Demand|Industry|Chemicals":
#        (
#            "Emissions|CO2|Energy|Demand|Industry|Chemicals",
#            "Production|Chemicals",
#        ),
#    "Emission Intensity|CO2|Industrial Processes|Chemicals":
#        (
#            "Emissions|CO2|Industrial Processes|Chemicals",
#            "Production|Chemicals",
#        ),
#    "Emission Intensity|CO2|Energy and Industrial Processes|Chemicals":
#        (
#            [
#                "Emissions|CO2|Energy|Demand|Industry|Chemicals",
#                "Emissions|CO2|Industrial Processes|Chemicals",
#            ],
#            "Production|Chemicals",
#        ),
    
# cement
    "Energy Intensity|Industry|Non-Metallic Minerals|Cement":
        (
            "Final Energy|Industry|Non-Metallic Minerals|Cement",
            "Production|Non-Metallic Minerals|Cement",
        ),
    "Emission Intensity|CO2|Industrial Processes|Non-Metallic Minerals|Cement":
        (
            "Emissions|CO2|Industrial Processes|Cement",
            "Production|Non-Metallic Minerals|Cement",
        ),
    "Emission Intensity|CO2|Energy and Industrial Processes|Non-Metallic Minerals|Cement":
        (
            "Emissions|CO2|Industrial Processes|Cement",
            "Production|Non-Metallic Minerals|Cement",
        ),
}

intensity_args = dict()

for key, value in tagged_intensity_args.items():
    for s in species:
        for f in supply_fuel:
            intensity_args[replace_tag(key, s, f)] = [
                replace_tag(value[i], s, f) for i in range(len(value))
            ]

In [ ]:
def append_to_list(lst, item):
    if pyam.utils.is_list_like(item):
        [append_to_list(lst, x) for x in item]
    else:
        lst.append(item)

In [ ]:
def to_list(mapping):
    _list = list()
    for key, value in mapping.items():
        append_to_list(_list, key)
        append_to_list(_list, value)
    return list(set(_list))

In [ ]:
variable_list = to_list(intensity_args)

In [ ]:
variable_list

In [ ]:
df = pyam.read_ixmp4(platform, variable=variable_list, region="World")

In [ ]:
df.variable

In [ ]:
df_list = []

for key, value in intensity_args.items():
    value_list = list()
    append_to_list(value_list, value)
    if any([v not in df.variable for v in value_list]):
        print("Skip " + key)
        continue
    
    df_list.append(
        df.divide(value[0], value[1], key, ignore_units=f"{df.filter(variable=value[0]).unit}/{df.filter(variable=value[1]).unit}")
    )

In [ ]:
result_df = pyam.concat(df_list)

In [ ]:
result_df.variable

In [ ]:
result_df.rename(
    unit={
        "['Mt CO2-equiv/yr']/['EJ/yr']": "Mt CO2-equiv/EJ",
        "['Mt CO2/yr']/['EJ/yr']": "Mt CO2/EJ",
        "['Mt CO2/yr']/['Mt/yr']": "Mt CO2/Mt",
        "['EJ/yr']/['Mt/yr']": "EJ/Mt",
    },
    inplace=True
)

In [ ]:
result_df.variable

In [ ]:
definition.validate(result_df)

In [ ]:
v = result_df.variable[3]

result_df.filter(variable=v, year=range(2020, 2101), region="World").plot()

In [ ]:
df_energy = pyam.read_ixmp4(platform, variable="Final Energy|Industry|Iron and Steel|Electricity", region="World")

In [ ]:
scope_df = result_df.append(df_energy).multiply(
    "Emission Intensity|CO2|Energy|Supply|Electricity", 
    "Final Energy|Industry|Iron and Steel|Electricity",
    "Emissions|CO2|Energy|Indirect Demand]|Industry|Iron and Steel",
    ignore_units="Mt CO2/yr",
)

In [ ]:
scope_df.append(
    df.add(
        "Emissions|CO2|Energy|Demand|Industry|Iron and Steel",
        "Emissions|CO2|Industrial Processes|Iron and Steel",
        "Emissions|CO2|Energy and Industrial Processes|Iron and Steel",
    ),
    inplace=True,
)

In [ ]:
scope_df.add(
    "Emissions|CO2|Energy and Industrial Processes|Iron and Steel",
    "Emissions|CO2|Energy|Indirect Demand]|Industry|Iron and Steel",
    "Emissions|CO2|Energy and Industrial Processes [incl. Indirect Demand]|Iron and Steel",
    append=True,
)

In [ ]:
for model, scenario in result_df.index:
    run = platform.runs.get(model, scenario)
    _df = result_df.filter(model=model, scenario=scenario)
    with run.transact("Add emissions- and energy-intensity for energy and industry (scope 1)"):
        run.iamc.add(_df.data)
        print(f"Done {model} | {scenario}")

In [ ]:
for model, scenario in scope_df.index:
    run = platform.runs.get(model, scenario)
    _df = result_df.filter(model=model, scenario=scenario)
    with run.transact("Add variables for indirect emissions"):
        run.iamc.add(_df.data)
        print(f"Done {model} | {scenario}")